In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import yfinance as yf
import datetime
import tensorflow as tf

#np.random.seed(42)
#tf.random.set_seed(42)

In [2]:
stock = "ICICIBANK"
data = yf.Ticker(f'{stock}.NS')
enddate = datetime.date.today() + datetime.timedelta(days = 1)
startdate = enddate - datetime.timedelta(days = 365 * 1) 
#open_prices = list(data.history(start = startdate, end = enddate).Open)
#high_prices = list(data.history(start = startdate, end = enddate).High)
#low_prices = list(data.history(start = startdate, end = enddate).Low)
close_prices = list(data.history(start = startdate, end = enddate).Close)
volume = list(data.history(start = startdate, end = enddate).Volume)
data.history(start = startdate, end = enddate).Close

Date
2023-02-20 00:00:00+05:30     845.143311
2023-02-21 00:00:00+05:30     847.226257
2023-02-22 00:00:00+05:30     832.398254
2023-02-23 00:00:00+05:30     832.894165
2023-02-24 00:00:00+05:30     832.844604
                                ...     
2024-02-12 00:00:00+05:30     996.099976
2024-02-13 00:00:00+05:30    1019.799988
2024-02-14 00:00:00+05:30    1022.700012
2024-02-15 00:00:00+05:30    1016.650024
2024-02-16 00:00:00+05:30    1023.250000
Name: Close, Length: 244, dtype: float64

In [3]:
# Convert the list to a NumPy array
close_prices = np.array(close_prices).reshape(-1, 1)

# Normalize/Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
prices_scaled = scaler.fit_transform(close_prices)

# Prepare data for LSTM
def create_sequences(data, seq_length):
    input_data, target = [], []
    for i in range(len(data) - seq_length - 1):
        input_data.append(data[i:(i + seq_length), 0])
        target.append(data[i + seq_length, 0])
    return np.array(input_data), np.array(target)

sequence_length = 20  # Change this to set the sequence length
x, y = create_sequences(prices_scaled, sequence_length)

# Reshape data for LSTM [samples, time steps, features]
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x, y, epochs=50, batch_size=32)

# Predict the next day's closing price
last_sequence = prices_scaled[-sequence_length:].reshape(1, sequence_length, 1)
predicted_price = model.predict(last_sequence)

# Inverse transform the predicted price to get the actual value
predicted_price = scaler.inverse_transform(predicted_price)

Epoch 1/50
7/7 [==============================] - 1s 8ms/step - loss: 0.1449
Epoch 2/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0324
Epoch 3/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0181
Epoch 4/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0180
Epoch 5/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0127
Epoch 6/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0113
Epoch 7/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0125
Epoch 8/50
7/7 [==============================] - 0s 7ms/step - loss: 0.0118
Epoch 9/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0126
Epoch 10/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0114
Epoch 11/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0105
Epoch 12/50
7/7 [==============================] - 0s 6ms/step - loss: 0.0104
Epoch 13/50
7/7 [==============================] - 0s 6ms/step - loss: 0.

In [4]:
train_loss = model.evaluate(x, y)
print("Training Loss:", train_loss)

7/7 [==============================] - 0s 2ms/step - loss: 0.0047
Training Loss: 0.004733739886432886


In [5]:
print("Predicted next day's closing price:", predicted_price[0, 0])

Predicted next day's closing price: 1012.0623
